# 3、分析

已经建立了群体用户画像，就在可视化分析的后面

然后我需要对这些csv求一些基本统计量之类的东西

## 3.1 用户画像-基本统计量

In [1]:
import os
import pandas as pd
import json
from datetime import datetime
from collections import Counter
import time
from tqdm import tqdm


In [2]:
# 10G
csv_dir = "10G_data_new/sum"

# 定义一个列表来收集每个CSV文件的统计信息
summary_stats = []

# 遍历目录下所有CSV文件
for filename in os.listdir(csv_dir):
    if filename.endswith(".csv"):
        file_path = os.path.join(csv_dir, filename)
        try:
            df = pd.read_csv(file_path)
            print(file_path)
            if "key" not in df.columns or "count" not in df.columns:
                continue  # 跳过格式不对的文件
            
            df["count"] = pd.to_numeric(df["count"], errors="coerce").fillna(0)
            total_count = df["count"].sum()
            unique_keys = df["key"].nunique()
            max_row = df.loc[df["count"].idxmax()]
            min_row = df.loc[df["count"].idxmin()]
            top_3_sum = df.nlargest(3, "count")["count"].sum()
            top_3_ratio = top_3_sum / total_count if total_count else 0

            summary_stats.append({
                "filename": filename,
                "total_count": total_count,
                "unique_keys": unique_keys,
                "max_key": max_row["key"],
                "max_count": max_row["count"],
                "min_key": min_row["key"],
                "min_count": min_row["count"],
                "top_3_ratio": round(top_3_ratio, 4)
            })
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# 转换为DataFrame
summary_df = pd.DataFrame(summary_stats)

print(summary_df)
# 保存输出结果
output_path = "10G_data_new/sum/user_profile_summary.csv"
summary_df.to_csv(output_path, index=False)


10G_data_new/sum/purchase_category_stats.csv
10G_data_new/sum/login_first_date_stats.csv
10G_data_new/sum/login_location_stats.csv
10G_data_new/sum/email_domain_bins.csv
10G_data_new/sum/timestamp_bins.csv
10G_data_new/sum/income_bins.csv
10G_data_new/sum/purchase_date_stats.csv
10G_data_new/sum/login_device_stats.csv
10G_data_new/sum/country_bins.csv
10G_data_new/sum/purchase_payment_method_stats.csv
10G_data_new/sum/age_bins.csv
10G_data_new/sum/address_province_bins.csv
10G_data_new/sum/purchase_status_stats.csv
10G_data_new/sum/avg_price_bins.csv
10G_data_new/sum/phone_code_bins.csv
10G_data_new/sum/gender_bins.csv
10G_data_new/sum/login_count_bins.csv
10G_data_new/sum/login_count_stats.csv
                             filename  total_count  unique_keys      max_key  \
0         purchase_category_stats.csv     45000000           42           耳机   
1          login_first_date_stats.csv     45000000         1910   2025-03-22   
2            login_location_stats.csv     71663185      

In [3]:
# 30G
csv_dir = "30G_data_new/sum"

# 定义一个列表来收集每个CSV文件的统计信息
summary_stats = []

# 遍历目录下所有CSV文件
for filename in os.listdir(csv_dir):
    if filename.endswith(".csv"):
        file_path = os.path.join(csv_dir, filename)
        try:
            df = pd.read_csv(file_path)
            print(file_path)
            if "key" not in df.columns or "count" not in df.columns:
                continue  # 跳过格式不对的文件
            
            df["count"] = pd.to_numeric(df["count"], errors="coerce").fillna(0)
            total_count = df["count"].sum()
            unique_keys = df["key"].nunique()
            max_row = df.loc[df["count"].idxmax()]
            min_row = df.loc[df["count"].idxmin()]
            top_3_sum = df.nlargest(3, "count")["count"].sum()
            top_3_ratio = top_3_sum / total_count if total_count else 0

            summary_stats.append({
                "filename": filename,
                "total_count": total_count,
                "unique_keys": unique_keys,
                "max_key": max_row["key"],
                "max_count": max_row["count"],
                "min_key": min_row["key"],
                "min_count": min_row["count"],
                "top_3_ratio": round(top_3_ratio, 4)
            })
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# 转换为DataFrame
summary_df = pd.DataFrame(summary_stats)

print(summary_df)
# 保存输出结果
output_path = "30G_data_new/sum/user_profile_summary.csv"
summary_df.to_csv(output_path, index=False)


30G_data_new/sum/purchase_category_stats.csv
30G_data_new/sum/login_first_date_stats.csv
30G_data_new/sum/login_location_stats.csv
30G_data_new/sum/email_domain_bins.csv
30G_data_new/sum/timestamp_bins.csv
30G_data_new/sum/income_bins.csv
30G_data_new/sum/purchase_date_stats.csv
30G_data_new/sum/login_device_stats.csv
30G_data_new/sum/country_bins.csv
30G_data_new/sum/purchase_payment_method_stats.csv
30G_data_new/sum/age_bins.csv
30G_data_new/sum/address_province_bins.csv
30G_data_new/sum/purchase_status_stats.csv
30G_data_new/sum/avg_price_bins.csv
30G_data_new/sum/phone_code_bins.csv
30G_data_new/sum/gender_bins.csv
30G_data_new/sum/login_count_bins.csv
30G_data_new/sum/login_count_stats.csv
                             filename  total_count  unique_keys     max_key  \
0         purchase_category_stats.csv    135000000           42          耳机   
1          login_first_date_stats.csv    135000000         1910  2025-03-23   
2            login_location_stats.csv    215003214         

## 3.2 用户行为分析

分析的核心字段（行为相关）


| 字段 | 描述 | 可分析行为 | 
| ---- | ---- | --------- | 
| last_login, registration_date | 时间戳 | 生命周期、活跃时间、冷启动用户 | 
| login_history（JSON） | 登录次数、设备、时长、登录时间分布 | 登录频率、日活跃、设备偏好 | 
| purchase_history（JSON） | 商品类别、支付、价格、时间 | 消费能力、购物频率、偏好模式 | 
| is_active | 用户状态 | 用户活跃度、唤醒策略 | 

最终希望对于每个用户，统计这些比较关键的信息，用于刻画用户的行为

| 特征名 | 含义 |
| ----- | ----- |
| login_days_count | 登录天数 |
| most_used_device | 最常用设备 |
| purchase_frequency | 购买频次 |
| avg_price | 平均消费金额 |
| top_category | 最常购买的商品类别 |

In [4]:
def extract_and_save_behavior_features(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    parquet_files = [f for f in os.listdir(input_folder) if f.endswith(".parquet")]

    for file in parquet_files:
        file_path = os.path.join(input_folder, file)
        print(f"\n正在处理: {file_path}")
        start_time = time.time()

        # df = pd.read_parquet(file_path)    # 太慢了，简化读取
        df = pd.read_parquet(file_path, columns=["id", "fullname", "login_history", "purchase_history"])

        result_rows = []

        for row in tqdm(df.itertuples(index=False), total=len(df), desc="行为分析", ncols=80, leave=False):
            user_id = row.id
            fullname = row.fullname

            # 登录记录
            login_history = row.login_history or {}
            if isinstance(login_history, str):
                try:
                    login_history = json.loads(login_history)
                except:
                    login_history = {}

            # 购买记录
            purchase_history = row.purchase_history or {}
            if isinstance(purchase_history, str):
                try:
                    purchase_history = json.loads(purchase_history)
                except:
                    purchase_history = {}

            # 登录行为
            login_days = login_history.get("login_count")
            devices = login_history.get("devices", [])
            if devices:
                most_common_device = Counter(devices).most_common(1)[0][0]

            # 购买行为
            avg_price = purchase_history.get("avg_price")
            category = purchase_history.get("categories")
            pay_method = purchase_history.get("payment_method")
            pay_status = purchase_history.get("payment_status")

            result_rows.append({
                "id": user_id,
                "fullname": fullname,
                "login_days": login_days,
                "most_common_device": most_common_device,
                "avg_price": avg_price,
                "purchase_category": category,
                "payment_method": pay_method,
                "payment_status": pay_status,
            })

        # 保存当前文件的结果
        output_csv = os.path.join(output_folder, file.replace(".parquet", ".csv"))
        pd.DataFrame(result_rows).to_csv(output_csv, index=False)

        end_time = time.time()
        print(f"完成处理: {file_path}，耗时 {end_time - start_time:.2f}s，保存为：{output_csv}")
        

In [5]:
# 用之前采样的1000条测试
folder = "10G_sample"
csv_folder = "10G_sample_csv"
start_time = time.time()
extract_and_save_behavior_features(folder, csv_folder)
end_time = time.time()
print(f"对于目录{folder}，执行时间为{(end_time-start_time):.2f}s")



正在处理: 10G_sample/parquet-00000-1000.parquet


完成处理: 10G_sample/parquet-00000-1000.parquet，耗时 0.04s，保存为：10G_sample_csv/parquet-00000-1000.csv
对于目录10G_sample，执行时间为0.05s


发现没问题，执行10G

In [6]:
# 10G
folder = "10G_data_new"
csv_folder = "10G_data_new_csv"
start_time = time.time()
extract_and_save_behavior_features(folder, csv_folder)
end_time = time.time()
print(f"对于目录{folder}，执行时间为{(end_time-start_time):.2f}s")



正在处理: 10G_data_new/part-00002.parquet


完成处理: 10G_data_new/part-00002.parquet，耗时 614.58s，保存为：10G_data_new_csv/part-00002.csv

正在处理: 10G_data_new/part-00006.parquet


完成处理: 10G_data_new/part-00006.parquet，耗时 614.78s，保存为：10G_data_new_csv/part-00006.csv

正在处理: 10G_data_new/part-00004.parquet


完成处理: 10G_data_new/part-00004.parquet，耗时 620.74s，保存为：10G_data_new_csv/part-00004.csv

正在处理: 10G_data_new/part-00000.parquet


完成处理: 10G_data_new/part-00000.parquet，耗时 606.82s，保存为：10G_data_new_csv/part-00000.csv

正在处理: 10G_data_new/part-00003.parquet


完成处理: 10G_data_new/part-00003.parquet，耗时 619.71s，保存为：10G_data_new_csv/part-00003.csv

正在处理: 10G_data_new/part-00001.parquet


完成处理: 10G_data_new/part-00001.parquet，耗时 618.41s，保存为：10G_data_new_csv/part-00001.csv

正在处理: 10G_data_new/part-00007.parquet


完成处理: 10G_data_new/part-00007.parquet，耗时 630.98s，保存为：10G_data_new_csv/part-00007.csv

正在处理: 10G_data_new/part-00005.parquet


完成处理: 10G_data_new/part-00005.parquet，耗时 629.43s，保存为：10G_data_new_csv/part-00005.csv
对于目录10G_data_new，执行时间为4966.14s


后来发现不对，这样太慢了，执行了一波加速，仅选取有用的列读取，这样会更快

In [6]:
# 30G
folder = "30G_data_new"
csv_folder = "30G_data_new_csv"
start_time = time.time()
extract_and_save_behavior_features(folder, csv_folder)
end_time = time.time()
print(f"对于目录{folder}，执行时间为{(end_time-start_time):.2f}s")



正在处理: 30G_data_new/part-00008.parquet


完成处理: 30G_data_new/part-00008.parquet，耗时 235.77s，保存为：30G_data_new_csv/part-00008.csv

正在处理: 30G_data_new/part-00013.parquet


完成处理: 30G_data_new/part-00013.parquet，耗时 249.62s，保存为：30G_data_new_csv/part-00013.csv

正在处理: 30G_data_new/part-00002.parquet


完成处理: 30G_data_new/part-00002.parquet，耗时 272.03s，保存为：30G_data_new_csv/part-00002.csv

正在处理: 30G_data_new/part-00015.parquet


完成处理: 30G_data_new/part-00015.parquet，耗时 280.68s，保存为：30G_data_new_csv/part-00015.csv

正在处理: 30G_data_new/part-00010.parquet


...

完成处理: 30G_data_new/part-00000.parquet，耗时 258.99s，保存为：30G_data_new_csv/part-00000.csv

正在处理: 30G_data_new/part-00003.parquet


完成处理: 30G_data_new/part-00003.parquet，耗时 255.29s，保存为：30G_data_new_csv/part-00003.csv

正在处理: 30G_data_new/part-00001.parquet


完成处理: 30G_data_new/part-00001.parquet，耗时 258.69s，保存为：30G_data_new_csv/part-00001.csv

正在处理: 30G_data_new/part-00007.parquet


完成处理: 30G_data_new/part-00007.parquet，耗时 264.18s，保存为：30G_data_new_csv/part-00007.csv

正在处理: 30G_data_new/part-00005.parquet


完成处理: 30G_data_new/part-00005.parquet，耗时 260.21s，保存为：30G_data_new_csv/part-00005.csv
对于目录30G_data_new，执行时间为4177.96s


## 3.3 高价值用户挖掘

然后要分析一下高价值用户

太多了，所以正常用到的聚类分析不是很建议，或者说，可以使用随机采样聚类发掘高价值评价标准，然后再寻找

或者说，直接使用打分机制

 |特征 | 条件 | 得分 | 说明 |
 | -- | ----- | ---- | ---- |
 |login_days | > 60 | +2 | 活跃用户 |
 |login_days | 30 ~ 60 | +1 | 中活跃 |
 |avg_price | > 5000 | +2 | 消费力高 |
 |avg_price | 3000 ~ 5000 | +1 | 中消费 |
 |income | >500000 | +1 | 高收入 |
 |payment_status | 已支付 | +1 | 有效交易 |
 |payment_status | 部分退款 / 已退款 | -1 | 有争议 |
 |most_common_device | 不加分 | 0 | 可选 |
 |purchase_category | 某些高价值类目（如数码） | +1 | 可选 |

In [10]:
def score_user(login_days, avg_price, income, payment_status):
    score = 0
    # 登录天数评分
    if login_days > 60:
        score += 2
    elif login_days > 30:
        score += 1
    # 平均价格评分
    if avg_price > 5000:
        score += 2
    elif avg_price > 3000:
        score += 1
    # 收入
    if income > 500000:
        score += 1
    # 支付状态评分
    if payment_status == "已支付":
        score += 1
    elif payment_status in {"部分退款", "已退款"}:
        score -= 1
    return score

In [16]:
def extract_high_value_ids(input_folder, output_folder, score_threshold=4):
    os.makedirs(output_folder, exist_ok=True)
    files = [f for f in os.listdir(input_folder) if f.endswith(".parquet")]

    for file in files:
        start_time = time.time()
        file_path = os.path.join(input_folder, file)
        print(f"\n正在处理: {file_path}")
        df = pd.read_parquet(file_path, columns=["id", "income", "login_history", "purchase_history", "fullname", "gender"], engine="pyarrow")
        
        high_value_ids = []

        for row in tqdm(df.itertuples(index=False), total=len(df), desc="筛选高价值", ncols=80):
            # 登录记录
            login_history = row.login_history or {}
            if isinstance(login_history, str):
                try:
                    login_history = json.loads(login_history)
                except:
                    login_history = {}

            # 购买记录
            purchase_history = row.purchase_history or {}
            if isinstance(purchase_history, str):
                try:
                    purchase_history = json.loads(purchase_history)
                except:
                    purchase_history = {}

            # 登录天数
            login_days = login_history.get("login_count")

            # 购买价格
            avg_price = purchase_history.get("avg_price")

            # 收入
            income = row.income

            login_days = login_history.get("login_count")
            avg_price = purchase_history.get("avg_price")
            payment_status = purchase_history.get("payment_status")

            score = score_user(login_days, avg_price, income, payment_status)

            if score >= score_threshold:
                high_value_ids.append(row.id)
        
        print(f"总计{len(high_value_ids)}名高价值用户")
        output_path = os.path.join(output_folder, file.replace(".parquet", ".csv"))
        pd.DataFrame({"id": high_value_ids}).to_csv(output_path, index=False)
        print(f"已保存高价值用户到: {output_path}")

        end_time = time.time()
        print(f"用时{(end_time-start_time):.2f}s")


In [17]:
# 测试小样本跑程序
folder = "10G_sample"
extract_high_value_ids(folder, folder)



正在处理: 10G_sample/parquet-00000-1000.parquet


筛选高价值:   0%|                                      | 0/1000 [00:00<?, ?it/s]

筛选高价值: 100%|████████████████████████| 1000/1000 [00:00<00:00, 43378.88it/s]

总计268名高价值用户
已保存高价值用户到: 10G_sample/parquet-00000-1000.csv
用时0.04s


In [18]:
# 10G
folder = "10G_data_new"
extract_high_value_ids(folder, folder)



正在处理: 10G_data_new/part-00002.parquet


筛选高价值: 100%|██████████████████| 5625000/5625000 [01:21<00:00, 69340.00it/s]


总计1533736名高价值用户
已保存高价值用户到: 10G_data_new/part-00002.csv
用时102.91s

正在处理: 10G_data_new/part-00006.parquet


筛选高价值: 100%|██████████████████| 5625000/5625000 [01:25<00:00, 65717.01it/s]


总计1533133名高价值用户
已保存高价值用户到: 10G_data_new/part-00006.csv
用时115.45s

正在处理: 10G_data_new/part-00004.parquet


...

总计1535606名高价值用户
已保存高价值用户到: 10G_data_new/part-00001.csv
用时105.84s

正在处理: 10G_data_new/part-00007.parquet


筛选高价值: 100%|██████████████████| 5625000/5625000 [01:22<00:00, 67967.18it/s]


总计1535312名高价值用户
已保存高价值用户到: 10G_data_new/part-00007.csv
用时105.97s

正在处理: 10G_data_new/part-00005.parquet


筛选高价值: 100%|██████████████████| 5625000/5625000 [01:21<00:00, 69120.32it/s]


总计1533328名高价值用户
已保存高价值用户到: 10G_data_new/part-00005.csv
用时105.59s


In [19]:
# 30G
folder = "30G_data_new"
extract_high_value_ids(folder, folder)



正在处理: 30G_data_new/part-00008.parquet


筛选高价值: 100%|██████████████████| 8437500/8437500 [02:01<00:00, 69386.50it/s]


总计2302888名高价值用户
已保存高价值用户到: 30G_data_new/part-00008.csv
用时152.95s

正在处理: 30G_data_new/part-00013.parquet


筛选高价值: 100%|██████████████████| 8437500/8437500 [02:07<00:00, 66307.64it/s]


总计2301143名高价值用户
已保存高价值用户到: 30G_data_new/part-00013.csv
用时163.45s

正在处理: 30G_data_new/part-00002.parquet


...

总计2301469名高价值用户
已保存高价值用户到: 30G_data_new/part-00001.csv
用时159.85s

正在处理: 30G_data_new/part-00007.parquet


筛选高价值: 100%|██████████████████| 8437500/8437500 [01:59<00:00, 70444.06it/s]


总计2301664名高价值用户
已保存高价值用户到: 30G_data_new/part-00007.csv
用时152.58s

正在处理: 30G_data_new/part-00005.parquet


筛选高价值: 100%|██████████████████| 8437500/8437500 [02:07<00:00, 66253.43it/s]


总计2300040名高价值用户
已保存高价值用户到: 30G_data_new/part-00005.csv
用时163.31s


因此就获得了10G和30G数据的高价值用户的id列表